# Temporal Graph Data Generation for 2-Camera MOT
In this notebook, we are going to generate the temporal graph dataset for the MOT task. Based on the previously preprocessed data the required manipulations are performed. A 2-camera scenario is considered. 

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
# pre-set the number of frames we want to look at 
n_frames = 10
n_cameras = 2

### 1. Load in the Preprocessed Data

In [3]:
with open('/home/ge93qew/WILDTRACK/Image_subsets/C1/reid-features/saved_dictionary-C1.pkl', 'rb') as fc1:
    loaded_dict_c1 = pickle.load(fc1)
    
with open('/home/ge93qew/WILDTRACK/Image_subsets/C2/reid-features/saved_dictionary-C2.pkl', 'rb') as fc2:
    loaded_dict_c2 = pickle.load(fc2)

We extract the re-id features and additional information from the loaded dictionaries. In order to split the data into several snapshots, we need to determine the number of elements corresponding to the same `frame_ID` for both cameras. 

In [4]:
re_id_c1, person_ID_c1, frame_ID_c1, name_c1, cam_ID_c1 = zip(*([loaded_dict_c1[i][0], loaded_dict_c1[i][1], 
                                                  loaded_dict_c1[i][2], loaded_dict_c1[i][3], 
                                                  loaded_dict_c1[i][4]] for i in loaded_dict_c1))
snapshot_dict_c1 = {int(i):frame_ID_c1.count(i) for i in frame_ID_c1}

In [5]:
re_id_c2, person_ID_c2, frame_ID_c2, name_c2, cam_ID_c2 = zip(*([loaded_dict_c2[i][0], loaded_dict_c2[i][1], 
                                                  loaded_dict_c2[i][2], loaded_dict_c2[i][3], 
                                                  loaded_dict_c2[i][4]] for i in loaded_dict_c2))
snapshot_dict_c2 = {int(i):frame_ID_c2.count(i) for i in frame_ID_c2}

### 3. Create Edge list and Edge Labels
Now, we have to think about which nodes shall be connected. This information shall be stored in an edge list in COO format. Here are some observations:
- Observation 1: there won't be any connection between new nodes from the same camera and time step, since a node represents a specific person and (s)he won't appear twice in the same frame 
- Observation 2: other than in the single-camera scenario, we do have edges for t=0, namely between all nodes from camera 1 and camera 2. 
- Observation 3: if the term historic nodes refers to nodes from past snapshots we need to connect every historic node with every new node. 

The most relevant information for defining the edges and their labels is the `person_ID`. An ideal graph model would label the edges between nodes that have the same `person_ID` with a `1` and all others with `0`. 

We first determine the number of cropped boxes for both cameras and investigate the graph evolution based on these numbers. 

In [6]:
num_cropped_boxes_c1 = []
for i in range(1,n_frames+1):
    num_cropped_boxes_c1.append(snapshot_dict_c1[i])
num_cropped_boxes_c1

[32, 31, 30, 33, 33, 33, 30, 30, 31, 31]

In [7]:
num_cropped_boxes_c2 = []
for i in range(1,n_frames+1):
    num_cropped_boxes_c2.append(snapshot_dict_c2[i])
num_cropped_boxes_c2

[18, 18, 17, 19, 18, 19, 18, 18, 19, 17]

In [8]:
num_nodes_c1 = [num_cropped_boxes_c1[0] if i == 1 else sum(num_cropped_boxes_c1[0:i]) for i in range(1,n_frames+1)]
num_nodes_c1

[32, 63, 93, 126, 159, 192, 222, 252, 283, 314]

In [9]:
num_nodes_c2 = [num_cropped_boxes_c2[0] if i == 1 else sum(num_cropped_boxes_c2[0:i]) for i in range(1,n_frames+1)]
num_nodes_c2

[18, 36, 53, 72, 90, 109, 127, 145, 164, 181]

In [10]:
num_cropped_boxes = [sum(value) for value in zip(num_cropped_boxes_c1, num_cropped_boxes_c2)]
num_cropped_boxes

[50, 49, 47, 52, 51, 52, 48, 48, 50, 48]

In [11]:
num_nodes_sum = [sum(value) for value in zip(num_nodes_c1, num_nodes_c2)]
num_nodes_sum

[50, 99, 146, 198, 249, 301, 349, 397, 447, 495]

Determine the number of overall edges for all graph snapshots. This information will be used later on.

In [12]:
summe = num_cropped_boxes_c1[0]*num_cropped_boxes_c2[0]
num_edges_per_snapshot = [summe]
print(summe)
for i in range(1,n_frames):
    tmp = num_cropped_boxes_c1[i]*num_cropped_boxes_c2[i] + num_nodes_sum[i-1]*(num_cropped_boxes[i])
    summe = summe + tmp
    print(f'summe: {summe}')
    num_edges_per_snapshot.append(summe)
summe

576
summe: 3584
summe: 8747
summe: 16966
summe: 27658
summe: 41233
summe: 56221
summe: 73513
summe: 93952
summe: 115935


115935

#### Create DataFrame Objects Containing the Mapping between Person's ID and Integer
Insert element 0 to first position in `num_nodes_c1` and `num_nodes_c2`.

In [13]:
num_nodes_c1.insert(0, 0)
num_nodes_c2.insert(0, 0)

In [14]:
num_nodes_c1

[0, 32, 63, 93, 126, 159, 192, 222, 252, 283, 314]

In [15]:
num_nodes_c2

[0, 18, 36, 53, 72, 90, 109, 127, 145, 164, 181]

Now create for both cameras as many `DataFrame` objects as specified in `n_frames`. We map a person's ID to a node ID to emphasize the node order. Particularly, this step assigns integer values to the node labels. Note that we introduced the variable `shift` in order to relatively calculate range boundaries for node IDs. Here is what we would like to have: 

- Node IDs that range from 0 to 31 for camera 1 nodes at t=0 (32 IDs)
- Node IDs that range from 32 to 49 for camera 2 nodes at t=0 (18 IDs)
- Node IDs that range from 50 to 80 for camera 1 nodes at t=1 (31 IDs)
- Node IDs that range form 81 to 98 for camera 2 nodes at t=1 (17 IDs)
- ...

Note that the integers in brackets correspond to the values in `num_cropped_boxes_c1` and `num_cropped_boxes_c2`. 

In [16]:
shift = 0
for frame in range(n_frames):
    for cam in range(n_cameras):
        exec(f'''global num_boxes,num_nodes,person_ID
num_boxes = num_cropped_boxes_c{cam+1}
num_nodes = num_nodes_c{cam+1}
person_ID = person_ID_c{cam+1}
''')
        nodeID_personID_dict = {'node_ID': range(shift,num_boxes[frame]+shift),
                               'person_ID': person_ID[num_nodes[frame]:num_nodes[frame+1]]}
        exec(f'''global tmp_df_c{cam+1}_{frame}
tmp_df_c{cam+1}_{frame} = pd.DataFrame(nodeID_personID_dict)
print(f'tmp_df_c{cam+1}_{frame}')
''') 
        shift = shift + num_boxes[frame]

tmp_df_c1_0
tmp_df_c2_0
tmp_df_c1_1
tmp_df_c2_1
tmp_df_c1_2
tmp_df_c2_2
tmp_df_c1_3
tmp_df_c2_3
tmp_df_c1_4
tmp_df_c2_4
tmp_df_c1_5
tmp_df_c2_5
tmp_df_c1_6
tmp_df_c2_6
tmp_df_c1_7
tmp_df_c2_7
tmp_df_c1_8
tmp_df_c2_8
tmp_df_c1_9
tmp_df_c2_9


#### Stack the Created DataFrames
For each timestep, we stack all existing dataframes camera-wise while preserving the node ordering we imposed. For instance, at t=2 we stack `tmp_df_c1_0`, `tmp_df_c1_1` and `tmp_df_c1_2` for camera 1 (similar for camera 2). 

In [17]:
df_c1_0 = tmp_df_c1_0
df_c2_0 = tmp_df_c2_0

dataframes_1 = [tmp_df_c1_0]
dataframes_2 = [tmp_df_c2_0]

for f in range(1,n_frames):
    exec(f'''global df_c1_{f}
global df_c2_{f}
dataframes_1.append(tmp_df_c1_{f})
dataframes_2.append(tmp_df_c2_{f})
df_c1_{f} = pd.concat(dataframes_1,ignore_index=True)
df_c2_{f} = pd.concat(dataframes_2,ignore_index=True)
''')

In [18]:
df_c1_9

,node_ID,person_ID
0,0,0000
1,1,0001
2,2,0002
3,3,0003
4,4,0004
...,...,...
309,473,0043
310,474,0044
311,475,0122
312,476,0151


In [19]:
for i in range(n_frames):
    for j in range(2):
        exec(f'''print(df_c{j+1}_{i}.shape)''')

(32, 2)
(18, 2)
(63, 2)
(36, 2)
(93, 2)
(53, 2)
(126, 2)
(72, 2)
(159, 2)
(90, 2)
(192, 2)
(109, 2)
(222, 2)
(127, 2)
(252, 2)
(145, 2)
(283, 2)
(164, 2)
(314, 2)
(181, 2)


#### Create DataFrame Objects Containing the Stacked Temporal Graph Data
We need to remove the first element we inserted at the beginning.

In [20]:
num_nodes_c1.pop(0)
num_nodes_c2.pop(0)

0

In [21]:
num_nodes_c1

[32, 63, 93, 126, 159, 192, 222, 252, 283, 314]

In [22]:
num_nodes_c2

[18, 36, 53, 72, 90, 109, 127, 145, 164, 181]

In [23]:
def generate_temporal_dataframes(num_frames): 
    df_stack = df_c1_0.merge(df_c2_0,how='cross')  
     
    for i in range(1,num_frames):       
        for c in range(1,n_cameras+1):
            gen_new_nodes_dfs_per_cam(c,i)

        # dataframe based on cartesian product of new nodes from both cameras c1 & c2 in each time step t > 0
        exec(f'''global df_new_edges_for_new_nodes
df_new_edges_for_new_nodes = new_nodes_c1.merge(new_nodes_c2,how='cross')
''')  
        # dataframe based on cartesian product of all new nodes and all historic nodes in each time step t > 0
        exec(f'''global df_new_edges_new_to_historic_nodes,new_nodes_c,df_c
new_nodes_c = pd.concat([new_nodes_c1,new_nodes_c2], ignore_index=True)
df_c = pd.concat([df_c1_{i-1},df_c2_{i-1}], ignore_index=True)
df_new_edges_new_to_historic_nodes = new_nodes_c.merge(df_c,how='cross') 
''')                  
        # dataframe containing the new set of historic nodes for the next iteration
        df_stack = pd.concat([df_stack,df_new_edges_for_new_nodes], ignore_index=True)  # -> correcly determined
        df_stack = pd.concat([df_stack,df_new_edges_new_to_historic_nodes], ignore_index=True)   
    return df_stack  

In [24]:
def gen_new_nodes_dfs_per_cam(cam,frame):
    exec(f'''global new_nodes_c{cam}
new_nodes_c{cam} = df_c{cam}_{frame}[num_nodes_c{cam}[frame-1]:num_nodes_c{cam}[frame]+1]
''')

In [25]:
b = generate_temporal_dataframes(n_frames)    
b

,node_ID_x,person_ID_x,node_ID_y,person_ID_y
0,0,0000,32,0002
1,0,0000,33,0005
2,0,0000,34,0010
3,0,0000,35,0012
4,0,0000,36,0013
...,...,...,...,...
115930,494,0383,442,0044
115931,494,0383,443,0122
115932,494,0383,444,0151
115933,494,0383,445,0247


In [26]:
#b.to_excel("dataframe_adjusted_node_ordering.xlsx") 

#### Create the Edge Labels
We first check if one node ID was used more than 1 time. Spoiler: all node IDs are unique.

In [27]:
b.loc[(b['node_ID_x'] == b['node_ID_y'])]

,node_ID_x,person_ID_x,node_ID_y,person_ID_y


If the node_IDs correspond to the same person ID set the corresponding edge label to `1`. Else to `0`.

In [28]:
b['labels'] = np.where((b['person_ID_x'] == b['person_ID_y']), 1, 0)
b

,node_ID_x,person_ID_x,node_ID_y,person_ID_y,labels
0,0,0000,32,0002,0
1,0,0000,33,0005,0
2,0,0000,34,0010,0
3,0,0000,35,0012,0
4,0,0000,36,0013,0
...,...,...,...,...,...
115930,494,0383,442,0044,0
115931,494,0383,443,0122,0
115932,494,0383,444,0151,0
115933,494,0383,445,0247,0


We have 3573 edges with the label `1`.

In [29]:
b.loc[b['labels'] == 1]

,node_ID_x,person_ID_x,node_ID_y,person_ID_y,labels
36,2,0002,32,0002,1
91,5,0005,33,0005,1
164,9,0010,34,0010,1
183,10,0012,35,0012,1
202,11,0013,36,0013,1
...,...,...,...,...,...
115860,494,0383,248,0383,1
115879,494,0383,300,0383,1
115897,494,0383,348,0383,1
115915,494,0383,396,0383,1


#### Generate the Edge List and Edge Label List
Based on the dataframe `b` that contains sub-dataframes that correspond to each snapshot we create the edge list and edge label list. Both lists contain numpy arrays each of which corresponds to one snapshot. In the process, we introduce the edge weight matrix and initialize it with all ones for each snapshot to indicate that we are dealing with unweighted graphs in each snapshot.

In [30]:
edge_indices = []
edge_labels = []
edge_weights = []

In [31]:
source_nodes = b['node_ID_x'].to_list()
target_nodes = b['node_ID_y'].to_list()

In [32]:
num_edges_per_snapshot

[576, 3584, 8747, 16966, 27658, 41233, 56221, 73513, 93952, 115935]

In [33]:
for i in range(0,n_frames):
    e_snap = np.transpose(np.array([list(a) for a in zip(source_nodes[0:num_edges_per_snapshot[i]], target_nodes[0:num_edges_per_snapshot[i]])]))
    edge_indices.append(e_snap)
    
    l_snap = np.array(b['labels'][0:num_edges_per_snapshot[i]])
    edge_labels.append(l_snap)
    
    edge_weights.append(np.ones(l_snap.size))

In [34]:
edge_indices

[array([[ 0,  0,  0, ..., 31, 31, 31],
        [32, 33, 34, ..., 47, 48, 49]]),
 array([[ 0,  0,  0, ..., 98, 98, 98],
        [32, 33, 34, ..., 47, 48, 49]]),
 array([[  0,   0,   0, ..., 145, 145, 145],
        [ 32,  33,  34, ...,  96,  97,  98]]),
 array([[  0,   0,   0, ..., 197, 197, 197],
        [ 32,  33,  34, ..., 143, 144, 145]]),
 array([[  0,   0,   0, ..., 248, 248, 248],
        [ 32,  33,  34, ..., 195, 196, 197]]),
 array([[  0,   0,   0, ..., 300, 300, 300],
        [ 32,  33,  34, ..., 246, 247, 248]]),
 array([[  0,   0,   0, ..., 348, 348, 348],
        [ 32,  33,  34, ..., 298, 299, 300]]),
 array([[  0,   0,   0, ..., 396, 396, 396],
        [ 32,  33,  34, ..., 346, 347, 348]]),
 array([[  0,   0,   0, ..., 446, 446, 446],
        [ 32,  33,  34, ..., 394, 395, 396]]),
 array([[  0,   0,   0, ..., 494, 494, 494],
        [ 32,  33,  34, ..., 444, 445, 446]])]

### 4. Extract Node Features and Node Labels
We start by recalling the number of total nodes in each snapshot as given in `num_nodes_c1` amd `num_nodes_c2`. We insert 0s again to these lists.

In [35]:
features = []
targets = [] # node labels

In [36]:
num_nodes_c1.insert(0, 0)
num_nodes_c2.insert(0, 0)

In [37]:
num_nodes_c1

[0, 32, 63, 93, 126, 159, 192, 222, 252, 283, 314]

In [38]:
num_nodes_c2

[0, 18, 36, 53, 72, 90, 109, 127, 145, 164, 181]

We now explot the node ordering we imposed at the beginning of this notebook.

In [39]:
features_snap = ()
features = []

targets_snap = []
targets = []

for i in range(n_frames):
    reid_c1 = re_id_c1[num_nodes_c1[i]:num_nodes_c1[i+1]]
    reid_c2 = re_id_c2[num_nodes_c2[i]:num_nodes_c2[i+1]]
    
    features_snap = features_snap + reid_c1 + reid_c2
    features.append(features_snap)
    
    target_c1 = np.array(person_ID_c1[num_nodes_c1[i]:num_nodes_c1[i+1]])
    target_c2 = np.array(person_ID_c2[num_nodes_c2[i]:num_nodes_c2[i+1]])
    
    targets_snap = np.concatenate([targets_snap,target_c1,target_c2])
    targets.append(targets_snap)

In [40]:
print(len(targets[9]))

495


### 5. Generate the Temporal Graph Dataset
We choose the iterator `DynamicGraphTemporalSignal` and pass the computed parameters.

In [46]:
from torch_geometric_temporal.signal import DynamicGraphTemporalSignal
from torch_geometric_temporal.signal import temporal_signal_split

In [42]:
tg_dataset = DynamicGraphTemporalSignal(edge_indices=edge_indices,
                                        edge_weights=edge_weights,
                                        features=features,
                                        targets=targets,
                                        edge_labels=edge_labels
            )

Splits iterator in the temporal dimension using a pre-set ratio. 

In [43]:
train_dataset, test_dataset = temporal_signal_split(tg_dataset, train_ratio=0.8)

In [47]:
for i in tg_dataset:
    print(i)

Data(x=[50, 2048], edge_index=[2, 576], edge_attr=[576], edge_labels=[576])
Data(x=[99, 2048], edge_index=[2, 3584], edge_attr=[3584], edge_labels=[3584])
Data(x=[146, 2048], edge_index=[2, 8747], edge_attr=[8747], edge_labels=[8747])
Data(x=[198, 2048], edge_index=[2, 16966], edge_attr=[16966], edge_labels=[16966])
Data(x=[249, 2048], edge_index=[2, 27658], edge_attr=[27658], edge_labels=[27658])
Data(x=[301, 2048], edge_index=[2, 41233], edge_attr=[41233], edge_labels=[41233])
Data(x=[349, 2048], edge_index=[2, 56221], edge_attr=[56221], edge_labels=[56221])
Data(x=[397, 2048], edge_index=[2, 73513], edge_attr=[73513], edge_labels=[73513])
Data(x=[447, 2048], edge_index=[2, 93952], edge_attr=[93952], edge_labels=[93952])
Data(x=[495, 2048], edge_index=[2, 115935], edge_attr=[115935], edge_labels=[115935])


In [45]:
import torch
#torch.save(tg_dataset, f'tg_dataset_2c-{n_frames}frames.pt')